파일 내부에 있는 엑셀에서 이메일 주소 추출 후 하나의 파일(total_email_adress.xlsx)로 저장

In [3]:
import os
import re
import openpyxl
from openpyxl import load_workbook

total_adress_list = [] # 찾는 목표
count2 = 0
# 엑셀 파일 찾기위해서 파일 이름들 보는 것 -> 완성
path_dir = os.getcwd()
file_list = os.listdir(path_dir)
ecxel_name_list = []
for i in file_list:
    var1 = i.find('.xlsx')
    if var1 < 0:
        continue
    ecxel_name_list.append(i)
# print(ecxel_name_list)

for i in ecxel_name_list:
    # 엑셀 파일에서 시트 이름 불러오기
    ecxel_adress = os.path.abspath(i) #파일의 주소 불러오는 것
    load_wb = load_workbook(ecxel_adress)
    sheet_name_list = load_wb.sheetnames
    # 찾은 sheet 이름을 이용해서 각 sheet 내의 이메일이 적히 열 찾기
    for j in sheet_name_list:
        # print(j)
        Row = 1
        Col = 1
        count = 0
        load_ws = load_wb[j]
        while 1:
            if Col < 20:
                Col = Col + 1
            else:
                Col = 1
                Row = Row + 1
            cell_val = str(load_ws.cell(Row, Col).value)
            var2 = cell_val.find('@')
            count = count + 1
            if var2 != -1 :
                break
            if count > 400:
                break
        # 찾은 열을 이용해서 각 주소 한 리스트에 넣기
        Row = 1
        count = 0
        old = count2
        while 1:
            cell_val = str(load_ws.cell(Row, Col).value)
            var3 = cell_val.find('@')
            if var3 != -1:
                total_adress_list.append(cell_val)
                count2 = count2 + 1
                count = 0
            elif var3 == -1:
                count = count+1
            Row = Row+1
            if count > 20:
                break
        #print(count2 - old) 

    # print(i) # 각 엑셀 명 확인
    # print(sheet_name_list) # 각 시트 명 확인
    # print(total_adress_list) # 총 저장된 이메일 주소 확인
    # print(len(total_adress_list)) # 총 이메일 주소 갯수 확인   
    
#중복 이메일 제거
total_adress_set = set(total_adress_list)
total_adress_list = list(total_adress_set)

# 셀이 여러개인 엑셀에서 주소값만 추출해서 리스트로 만드는 것 성공!

# 모은 이메일을 하나의 엑셀 파일에 정리
wb = openpyxl.Workbook()
sheet = wb.active
sheet.title = 'email_adress'

count3 = 1
for x in total_adress_list :
    sheet.cell(row = count3, column = 1).value = x
    count3 = count3 + 1 
wb.save('total_email_adress.xlsx')

# print('완료')
# print(total_adress_list)
# print(len(total_adress_list))

저장된 파일(total_email_adress.xlsx)을 이용하여 전체에게 메일 보내기 / 메일 계정 선택(id,password 입력)/ 제목 입력

In [5]:
# 메일 보낼 상대의 이메일 주소 리스트(you_list)를 만들기 위한 모듈
import os
import re
import openpyxl
from openpyxl import load_workbook

# 메일을 보내기 위한 모듈
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

user_id = input('아이디를 입력해주세요(네이버 아이디만 가능)')
user_password = input('비밀번호를 입력해주세요')
user_email = user_id+'@naver.com'
error_email_list = []

smtp_info = dict({"smtp_server" : "smtp.naver.com", # SMTP 서버 주소
                  "smtp_user_id" : user_id, 
                  "smtp_user_pw" : user_password,
                  "smtp_port" : 587}) # SMTP 서버 포트
                  
def send_email(smtp_info, msg):
    with smtplib.SMTP(smtp_info["smtp_server"], smtp_info["smtp_port"]) as server:
        # TLS 보안 연결
        server.starttls() 
        # 로그인
        server.login(smtp_info["smtp_user_id"], smtp_info["smtp_user_pw"])

        # 로그인 된 서버에 이메일 전송
        response = server.sendmail(msg['from'], msg['to'], msg.as_string()) # 메시지를 보낼때는 .as_string() 메소드를 사용해서 문자열로 바꿔줍니다.

        # 이메일을 성공적으로 보내면 결과는 {}
        if not response:
            print('이메일을 성공적으로 보냈습니다.')
        else:
            print(response)
  
               
# me == my email address
# you == recipient's email address
me = user_email
you_list = []

ecxel_adress = os.path.abspath('total_email_adress.xlsx') #파일의 주소 불러오는 것
load_wb = load_workbook(ecxel_adress)
load_ws = load_wb['email_adress']
order = 1
while 1 :
    cell_val = str(load_ws.cell(order,1).value)
    var2 = cell_val.find('@')
    var = cell_val.find('@')
    order = order + 1
    if var == -1 :
        break
    you_list.append(cell_val)
you_list.append(user_email)

# Create message container - the correct MIME type is multipart/alternative.
subject = input('메일 제목을 입력해주세요')
for j in you_list :
    msg = MIMEMultipart('alternative')
    msg['Subject'] = subject
    msg['From'] = me
    msg['To'] = j

    # Create the body of the message (a plain-text and an HTML version).
    html_start = """\
    <html>
      <head></head>
      <body>
      """

    html_final = """
      </body>
    </html>
    """

    filename = "index.txt"
    text=""
    with open (filename, 'rt', encoding='utf8') as myfile:  # 파일 불러내기
        for myline in myfile:                 # 모든 파일 1줄씩 읽기
            text = text+myline                # 1줄씩 출력하기
    html_index = text
    html = html_start + html_index + html_final


    #개선해야할 점 : html은 메모장에서 추출해서 사용하되 img파일만 따로
    #사용 방법은 네이버에서 메일보내는걸로 만든 후에 html 코드 복사 붙여넣기 메모장으로 해서 놓고 이미지 파일 놓으면 exe로 실행시켜서 메일 보내기
    #보내야하는 이메일 주소도 크롤링 해야하고 메일 주소 이상한건 제외하는 것도 해야하고 중복 메일도 체크해야할 듯 -> 완료
    #Record the MIME types of both parts - text/plain and text/html.
    #내게 보내기로 확인 후 전체에게 보내는게 좋을 듯
    part2 = MIMEText(html, 'html')

    # Attach parts into message container.
    # According to RFC 2046, the last part of a multipart message, in this case
    # the HTML message, is best and preferred.
    msg.attach(part2)
    try:
        send_email(smtp_info, msg)
    except:
        error_email_list.append(j)
if len(error_email_list) > 0 :
    print("에러가 난 이메일 목록")
    print(error_email_list)

out = input('종료하고 싶으면 엔터를 눌러주세요')
print('종료')

아이디를 입력해주세요(네이버 아이디만 가능)chunccc1004
비밀번호를 입력해주세요as2020@@
메일 제목을 입력해주세요Test3
이메일을 성공적으로 보냈습니다.
이메일을 성공적으로 보냈습니다.
이메일을 성공적으로 보냈습니다.
이메일을 성공적으로 보냈습니다.
에러가 난 이메일 목록
['ㅏㅣㅇㄴ러ㅏㅣㄴㅇ@ㅏㅣㄹ어니ㅏ']
